In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## USA | most popular genres
- absolute number of tracks sold in the US
- US purchases share of all purchases
- US purchases share of genre purchases

In [6]:
%%sql
WITH 
invoice_line_genre_customer AS (
    SELECT 
        il.invoice_line_id,
        il.track_id,
        g.genre_id, 
        g.name genre_name, 
        c.country customer_country
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
),
genre_tracks_sold AS (
    SELECT 
        ilg.genre_id,
        ilg.genre_name, 
        COUNT(ilg.invoice_line_id) tracks_sold
    FROM invoice_line_genre_customer ilg
    GROUP BY ilg.genre_id, ilg.genre_name
),
genre_tracks_sold_usa AS (
    SELECT 
        ilg.genre_id, 
        ilg.genre_name, 
        COUNT(ilg.invoice_line_id) tracks_sold_usa
    FROM invoice_line_genre_customer ilg
    WHERE ilg.customer_country = 'USA'
    GROUP BY ilg.genre_id, ilg.genre_name
)

SELECT 
    usa.genre_name, 
    usa.tracks_sold_usa,
    ROUND(CAST(usa.tracks_sold_usa AS FLOAT) * 100 / (
            SELECT SUM(tracks_sold_usa) 
            FROM genre_tracks_sold_usa
        ), 2) tracks_sold_usa_pct,
    ROUND(CAST(usa.tracks_sold_usa AS FLOAT) * 100 / world.tracks_sold, 1) tracks_sold_usa_genre_pct
    
FROM genre_tracks_sold_usa usa
INNER JOIN genre_tracks_sold world ON world.genre_id = usa.genre_id
GROUP BY 1
ORDER BY tracks_sold_usa DESC

Done.


genre_name,tracks_sold_usa,tracks_sold_usa_pct,tracks_sold_usa_genre_pct
Rock,561,53.38,21.3
Alternative & Punk,130,12.37,26.4
Metal,124,11.8,20.0
R&B/Soul,53,5.04,33.3
Blues,36,3.43,29.0
Alternative,35,3.33,29.9
Latin,22,2.09,13.2
Pop,22,2.09,34.9
Hip Hop/Rap,20,1.9,60.6
Jazz,14,1.33,11.6


### Interpretation

Of the new artist entries *Red Tone* should appeal to many Americans as the genre **Alternative & Punk** is one of the top-selling genres in the USA. The risk of most sales attributing to the *Alternative* in *Alternative & Punk* is mitigated by the fact that *Alternative* also exists as its own genre entry.

More than every second **Hip Hop/Rap** track sold has a buyer from the USA (60.6%). *Hip Hop/Rap* tracks sold in the USA, however, make up only about 2% of all track sales which is only marginally less than *Pop* genre's sales with only 35% American buyers. *Regal* should therefore make an interesting addition to the store tailored to American buyers.

About 60% more **Blues** tracks are sold to American buyers than *Hip Hop/Rap* and *Pop* tracks therefore *Slim Jim Bites* should be included in our store.

**Pop** music neither has high sale numbers nor a high share of American buyers on our store which makes *Meteor and the Girls* the least favorable pick.

**Recommended picks**
1. *Red Tone*
2. *Slim Jim Bites*
3. *Regal*


## Sales Support Performance

In [36]:
%%sql
SELECT 
    e.first_name||' '||e.last_name support_rep_name,
    COUNT(DISTINCT c.customer_id) assigned_customers,
    ROUND(SUM(i.total) / COUNT(DISTINCT c.customer_id), 2) customer_avg_value,
    DATE(e.hire_date) hire_date,
    ROUND(SUM(i.total), 2) total_sales
FROM customer c
INNER JOIN employee e ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY support_rep_name

Done.


support_rep_name,assigned_customers,customer_avg_value,hire_date,total_sales
Jane Peacock,21,82.45,2017-04-01,1731.51
Margaret Park,20,79.2,2017-05-03,1584.0
Steve Johnson,18,77.44,2017-10-17,1393.92


### Interpretation

Measured by the sum total of customer purchases, Jane Peacock who represents the largest number of customers performed best with 1731.51$ worth of customer purchases. Jane also has the highest average purchase value per customer which refutes the notion that total sales are caused by the higher number of customers.  

The number of assigned customers as well as the average purchase value per customer increases with the support representatives job tenure.

## Customer Countries

In [7]:
%%sql
WITH 
invoice_country AS (
    SELECT 
        i.total,
        c.customer_id,
        c.country country
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
),
country_customer_stats AS (
    SELECT 
        country,
        COUNT(DISTINCT customer_id) customers,
        SUM(total) value_of_sales, 
        SUM(total) / COUNT(DISTINCT customer_id) customer_avg_value,
        AVG(total) order_avg_value
    FROM invoice_country
    GROUP BY country
),
country_customer_stats_other AS (
    SELECT
        grouping,
        SUM(customers) customers,
        SUM(value_of_sales) value_of_sales,
        AVG(customer_avg_value) customer_avg_value,
        AVG(order_avg_value) order_avg_value
    FROM (
        SELECT *,
            CASE
                WHEN customers = 1 THEN 'Other'
                ELSE country
            END AS grouping
        FROM country_customer_stats
        )
    GROUP BY grouping
)

SELECT 
    grouping country,
    customers,
    ROUND(value_of_sales, 2) value_of_sales,
    ROUND(customer_avg_value, 2) customer_avg_value,
    ROUND(order_avg_value, 2) order_avg_value
FROM (
    SELECT *,
        CASE
            WHEN grouping = 'Other' THEN 1
            ELSE 0
        END AS sort
    FROM country_customer_stats_other
    )
ORDER BY sort, value_of_sales DESC

Done.


country,customers,value_of_sales,customer_avg_value,order_avg_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


### Album vs. Individual Track Purchases

In [ ]:
    SELECT 
        (
            SELECT track_id FROM invoice_tracks 
            EXCEPT 
            SELECT track_id FROM tracks t WHERE t.album_id = ia.album_id
        ) IS NULL
        AND
        SELECT (
            SELECT track_id FROM tracks t WHERE t.album_id = ia.album_id
            EXCEPT
            ia.track_id
        ) IS NULL full_album

In [91]:
%%sql
WITH
invoice_tracks AS (
    SELECT i.invoice_id, t.track_id, a.album_id
    FROM invoice i
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
    INNER JOIN album a ON a.album_id = t.album_id
),
invoice_full_album AS (
    SELECT 
        invoice_id,
        (
            SELECT i.total
            FROM invoice i
            WHERE i.invoice_id = it.invoice_id
        ) invoice_total,
        (
            SELECT track_id FROM invoice_tracks it2 WHERE it2.invoice_id = it.invoice_id
            EXCEPT
            SELECT track_id FROM track t WHERE t.album_id = it.album_id
        ) IS NULL 
        AND
        (
            SELECT track_id FROM track t WHERE t.album_id = it.album_id
            EXCEPT
            SELECT track_id FROM invoice_tracks it2 WHERE it2.invoice_id = it.invoice_id
        ) IS NULL full_album
    FROM invoice_tracks it
    GROUP BY invoice_id
)
SELECT
    CASE
        WHEN ifa.full_album = 1 THEN 'Album'
        ELSE 'Tracks'
    END order_type,
    COUNT(*) invoices,
    ROUND(CAST(COUNT(*) AS FLOAT) * 100 / (
        SELECT COUNT(*)
        FROM invoice
        ), 2) invoices_pct,
    ROUND(SUM(ifa.invoice_total),2) total_value,
    ROUND(SUM(ifa.invoice_total) * 100 / (
        SELECT SUM(invoice_total)
        FROM invoice_full_album
        ),2) total_value_pct
FROM invoice_full_album ifa
GROUP BY ifa.full_album

Done.


order_type,invoices,invoices_pct,total_value,total_value_pct
Tracks,500,81.43,3248.19,68.97
Album,114,18.57,1461.24,31.03


### Interpretation

81.5% of purchases are purchases of individual tracks rather than whole albums. These purchases make for 69% of total sales.
That means that 31% of total sales still come from whole album purchases.  
Further analysis could give insight into which albums were commonly bought in full and provide a basis for a mixed strategy where certain albums are provided in full while for ohters only the most popular tracks are picked.  

Currently the 